## Gaussian Beam

- The Strategy
- 
-

### The Strategy

The goal is to write a gaussian light source to measure attenuations<br>
through the lattice. Since only rays which exit the lattice from the<br>
rear plane will be measured, restricting the range of the generated <br>
rays to those which pass through this plane provides computational<br>
savings.<p>

My strategy for attaining these savings is as follows:
- Estimate the number of gaussian rays needed for 1M rays to land on a given plane.
- Create this gaussian distribution on the exit plane.
- Pullback the distribution (via rescaling) to the entrance plane.


In [7]:
import Data.Random.Normal
import Statistics.Distribution.Normal
import Statistics.Distribution

type Center = Double
type Deviation = Double

-- Given a center, deviation and possible value returns a probability
getProb :: Center -> Deviation -> Double -> Double 
getProb μ σ x = cumulative (normalDistr μ σ) x

-- Calculates needed Rays to have a high probability that
-- n rays are within -1 and 1 given deviation σ.
neededRays :: Double -> Deviation -> Int -- (10^6)
neededRays n σ = truncate $ n / probArea σ (-1, 1)
  where
    probArea σ (l, u) = getProb 0 σ 1 - getProb 0 σ (-1)


<interactive>:1:1: error:
    Failed to load interface for ‘Data.Random.Normal’
    Use -v to see a list of the files searched for.

The code above calculates the needed rays. From these calculations one can<br>
specify a gaussian distribution on the exit plane and have fairly good<br>
certainty that about 1 million rays will hit the photographic plate.<p>

This is where the dimensions of the given problem enter. The lattice itself<br>
has dimensions $1mm^3$ but the gaussian is specified by it's center and so<br>
its standard unit is $1/2mm$. To standardize, I decide to use the distributions<br>
units and set the lattice to be $2units^3$.<p>

The gaussian is determined on the exit plane via the distance from the light source<br>
and the given deviation. To transfer this distribution to the entrance plane requires<br>
some arithmetic.<br>

Given a distance d from source to input plane in standard units, `d + 2` gives the<br>
distance from source to exit plane in the same units. Setting `tan θ = 1 / (d + 2) = r / d`,<br>
where r is the radius on the input plane, the scaling factor from exit plane to the<br>
entrance plane is found to be: `r = d / (d + 2)`.



In [18]:
type Ray = (EntryCoords, EntryAngles)
type EntryAngles = (Double, Double)
type EntryCoords = (Double, Double)
type Distance = Double
type Center = Double
type Beam = [Ray]

center = 50

ray :: Distance -> EntryCoords -> Ray
ray d (x, z) = ((coords x (d/2), coords z (d/2)), (angles x d, angles z d))
  where
    -- @(x > 0, d == 0) => 1.1071487177940904
    -- @(x < 0, d == 0) => 2.0344439357957027
    coords t d = center * t * d / (d + 1) + center
    angles t d | t > 0 = atan ((d+2)/t)
               | otherwise = pi + atan ((d+2)/t)

Then using transport: transport cs as => Spew City
